# 01 — Setup & Data Fetch (TSLA, BND, SPY)

In [1]:
# === Colab Setup ===
!pip install -U pip --quiet
!pip install pandas numpy matplotlib seaborn scikit-learn statsmodels yfinance PyPortfolioOpt arch --quiet
import sys, os, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CWD:", os.getcwd())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00
Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
CWD: /content


In [2]:
# === Configuration ===
START_DATE = "2015-07-01"
END_DATE   = "2025-07-31"
TICKERS    = ["TSLA", "BND", "SPY"]
BT_START   = "2024-08-01"
BT_END     = "2025-07-31"
TRADING_DAYS = 252
RISK_FREE_DAILY = 0.0

DATA_DIR = "data"
OUT_PLOTS = "outputs/plots"
OUT_MODELS = "outputs/models"
OUT_METRICS = "outputs/metrics"

for d in [DATA_DIR, OUT_PLOTS, OUT_MODELS, OUT_METRICS]:
    os.makedirs(d, exist_ok=True)


In [3]:
# === Fetch from Yahoo Finance and save CSVs ===
import yfinance as yf, pandas as pd
data = {}
for tk in TICKERS:
    df = yf.download(tk, start=START_DATE, end=END_DATE, interval="1d", auto_adjust=False, progress=False)
    df.index.name = "Date"
    for c in ["Open","High","Low","Close","Adj Close","Volume"]:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[["Open","High","Low","Close","Adj Close","Volume"]]
    df.to_csv(f"{DATA_DIR}/{tk}_1d.csv")
    data[tk] = df

prices = pd.concat({k: v["Adj Close"] for k,v in data.items()}, axis=1)
prices.columns = TICKERS
prices.to_csv(f"{DATA_DIR}/prices_adj_close.csv")
returns = prices.pct_change().dropna()
returns.to_csv(f"{DATA_DIR}/returns_daily.csv")

print("Saved:", [f"{DATA_DIR}/{tk}_1d.csv" for tk in TICKERS] + [f"{DATA_DIR}/prices_adj_close.csv", f"{DATA_DIR}/returns_daily.csv"])
prices.tail()

Saved: ['data/TSLA_1d.csv', 'data/BND_1d.csv', 'data/SPY_1d.csv', 'data/prices_adj_close.csv', 'data/returns_daily.csv']


,TSLA,BND,SPY
Date,,,
2025-07-24,305.299988,72.698860,634.419983
2025-07-25,316.059998,72.878265,637.099976
2025-07-28,325.589996,72.758659,636.940002
2025-07-29,321.200012,73.137398,635.260010
2025-07-30,319.040009,72.948036,634.460022
